# BertViz Neuron View - Modello BERT Italiano

Questo notebook dimostra come utilizzare BertViz per visualizzare le attenzioni di un modello BERT italiano.

## Setup

Prima di tutto, assicuriamoci che tutte le dipendenze siano installate e configurate correttamente.

In [ ]:
# Import delle librerie necessarie
import sys
import os

# Aggiungiamo la directory del progetto al path
project_dir = os.path.dirname(os.path.abspath(''))
if project_dir not in sys.path:
    sys.path.insert(0, project_dir)

print(f"Project directory: {project_dir}")
print(f"Python version: {sys.version}")

In [ ]:
# Import delle librerie principali
import torch
from transformers import AutoTokenizer
from bertviz.neuron_view import show

# Import del nostro modello personalizzato
from modeling_bert_italian import BertModelIT

print("✓ Tutte le librerie importate con successo!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA disponibile: {torch.cuda.is_available()}")

## Caricamento del Modello BERT Italiano

Useremo il modello `dbmdz/bert-base-italian-xxl-cased`, uno dei migliori modelli BERT per la lingua italiana.

In [ ]:
# Nome del modello BERT italiano pre-addestrato
model_name = "dbmdz/bert-base-italian-xxl-cased"

print(f"Caricamento del modello: {model_name}")
print("Questo potrebbe richiedere alcuni minuti al primo avvio...")

# Carica il tokenizzatore standard
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Carica il nostro modello personalizzato con i pesi pre-addestrati
# IMPORTANTE: impostiamo output_attentions=True per ottenere le attenzioni
model = BertModelIT.from_pretrained(
    model_name, 
    output_attentions=True
)

# Impostiamo il modello in modalità evaluation
model.eval()

print("✓ Modello e tokenizzatore caricati con successo!")
print(f"Numero di parametri: {sum(p.numel() for p in model.parameters()):,}")

## Test del Modello con Testo Italiano

Ora testiamo il modello con alcune frasi in italiano per verificare che funzioni correttamente.

In [ ]:
# Frasi di test in italiano
sentence_a = "Il gatto si è seduto sul tappeto."
sentence_b = "Il cane dormiva sulla poltrona."

print("Frasi di test:")
print(f"  Frase A: {sentence_a}")
print(f"  Frase B: {sentence_b}")

# Tokenizziamo le frasi
inputs = tokenizer(
    sentence_a, 
    sentence_b, 
    return_tensors="pt",
    padding=True,
    truncation=True
)

print("\nTokens:")
print(f"  Input IDs shape: {inputs['input_ids'].shape}")
print(f"  Tokens: {tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])}")

In [ ]:
# Eseguiamo il forward pass per verificare che il modello funzioni
with torch.no_grad():
    outputs = model(**inputs)

print("Output del modello:")
print(f"  Last hidden state shape: {outputs.last_hidden_state.shape}")
print(f"  Numero di layer con attenzioni: {len(outputs.attentions)}")

if outputs.attentions:
    attn = outputs.attentions[0]
    print(f"  Shape delle attenzioni (layer 0): {attn.shape}")
    print(f"    (batch_size, num_heads, seq_length, seq_length)")
    
print("\n✓ Il modello funziona correttamente!")

## Visualizzazione con BertViz Neuron View

Ora utilizziamo BertViz per visualizzare le attenzioni del modello. La Neuron View mostra come i query e key vectors interagiscono per produrre i punteggi di attenzione.

**Parametri:**
- `layer`: il layer da visualizzare (0-11 per BERT base)
- `head`: la testa di attenzione da visualizzare (0-11 per BERT base)

In [ ]:
# Visualizzazione della Neuron View
# Layer 2, Head 8 (puoi modificare questi valori)
layer_to_view = 2
head_to_view = 8

print(f"Visualizzazione del layer {layer_to_view}, head {head_to_view}")
print("Nota: La visualizzazione apparirà qui sotto\n")

try:
    show(
        model, 
        "bert", 
        tokenizer, 
        sentence_a, 
        sentence_b, 
        layer=layer_to_view, 
        head=head_to_view
    )
    print("\n✓ Visualizzazione completata!")
except Exception as e:
    print(f"❌ Errore durante la visualizzazione: {e}")
    print(f"Tipo di errore: {type(e).__name__}")
    import traceback
    traceback.print_exc()

## Esempi Aggiuntivi

Proviamo con altre frasi italiane per esplorare diverse attenzioni.

In [ ]:
# Esempio 2: Frasi con relazioni grammaticali più complesse
sentence_a2 = "La studentessa che studia matematica ha superato l'esame."
sentence_b2 = "Il professore le ha fatto i complimenti."

print(f"Frase A: {sentence_a2}")
print(f"Frase B: {sentence_b2}\n")

show(model, "bert", tokenizer, sentence_a2, sentence_b2, layer=5, head=3)

In [ ]:
# Esempio 3: Frasi con termini tecnici
sentence_a3 = "L'intelligenza artificiale trasforma il modo in cui lavoriamo."
sentence_b3 = "Gli algoritmi di machine learning apprendono dai dati."

print(f"Frase A: {sentence_a3}")
print(f"Frase B: {sentence_b3}\n")

show(model, "bert", tokenizer, sentence_a3, sentence_b3, layer=8, head=5)

## Esplorazione Interattiva

Puoi modificare i seguenti parametri per esplorare diversi layer e head:

In [ ]:
# Parametri modificabili
YOUR_SENTENCE_A = "Scrivi qui la tua prima frase in italiano."
YOUR_SENTENCE_B = "Scrivi qui la tua seconda frase in italiano."
YOUR_LAYER = 6  # 0-11 per BERT base
YOUR_HEAD = 4   # 0-11 per BERT base

print(f"Visualizzazione personalizzata:")
print(f"  Layer: {YOUR_LAYER}")
print(f"  Head: {YOUR_HEAD}\n")

show(
    model, 
    "bert", 
    tokenizer, 
    YOUR_SENTENCE_A, 
    YOUR_SENTENCE_B, 
    layer=YOUR_LAYER, 
    head=YOUR_HEAD
)

## Informazioni sul Modello

Alcune informazioni utili sul modello BERT italiano utilizzato.

In [ ]:
print("Informazioni sul modello:")
print(f"  Nome: {model_name}")
print(f"  Numero di layer: {model.config.num_hidden_layers}")
print(f"  Numero di attention heads: {model.config.num_attention_heads}")
print(f"  Hidden size: {model.config.hidden_size}")
print(f"  Intermediate size: {model.config.intermediate_size}")
print(f"  Max position embeddings: {model.config.max_position_embeddings}")
print(f"  Vocabolario: {model.config.vocab_size} tokens")

## Note Tecniche

### Come Funziona

1. **BertSelfAttentionIT**: Modifica il metodo `forward` per restituire `query_layer` e `key_layer` oltre all'output standard.

2. **Propagazione**: Le modifiche si propagano attraverso `BertAttentionIT`, `BertLayerIT`, `BertEncoderIT` fino a `BertModelIT`.

3. **Compatibilità**: Il modello è completamente compatibile con i pesi pre-addestrati di Hugging Face.

### Parametri della Visualizzazione

- **Layer**: Ogni layer del transformer apprende rappresentazioni diverse. I layer iniziali tendono a catturare caratteristiche sintattiche, mentre i layer finali catturano semantica più astratta.

- **Head**: Ogni head di attenzione può specializzarsi in relazioni diverse (es. dipendenze sintattiche, co-riferimento, ecc.).

### Risorse

- [BertViz Documentation](https://github.com/jessevig/bertviz)
- [BERT Paper](https://arxiv.org/abs/1810.04805)
- [Modello BERT Italiano](https://huggingface.co/dbmdz/bert-base-italian-xxl-cased)